In [1]:
import random
import ast
import pandas as pd
import numpy as np
import re
import pickle

## Load Article Table

In [15]:
data = pd.read_csv('articles.csv') 
data = data.iloc[:,1:]
data.head()

,index,title,image,description,tags,host,duedate,uploaderid,offlineupload,uploaddate
0,0,한국야쿠르트 마케팅 공모전,https://s3.ap-northeast-2.amazonaws.com/media....,2020년 한국야쿠르트 마케팅 공모전 ■ 공모주제 · 언택트 시대에 걸맞은 차별...,"['공모전', '경영']",한국야쿠르트,2020-08-31,3,1,2020-09-04
1,1,행정안전부 제1회 도전.한국 대국민 아이디어 공모전,https://s3.ap-northeast-2.amazonaws.com/media....,제1회 도전.한국 대국민 아이디어 공모전▷ 지원자격 : 관심 있는 국민‧기업‧단체...,"['공모전', '봉사']",행정안전부,2020-09-15,10,1,2020-09-04
2,2,K-water 2020년 대국민 환경 빅데이터 공모전,https://s3.ap-northeast-2.amazonaws.com/media....,2020년 K-water 대국민 환경 빅데이터 공모전▷ 담당자 전화번호 :042-6...,"['공모전', 'IT', '과학']",k-water,2020-08-31,2,1,2020-09-04
3,3,기획재정부 국민참여예산 대학생 공모전,https://s3.ap-northeast-2.amazonaws.com/media....,[기획재정부] 국민참여예산 대학생 공모전 (~8/31)■ 신청기간2020년 6월 2...,"['공모전', '사회']",기획재정부,2020-08-31,10,0,2020-09-04
4,4,문학동네 2020 제26회 문학동네 소설상 공모,NaN,2020 제26회 문학동네 소설상 공모▷ 응모마감 : 2020년 8월 30일 (마감...,['공모전'],문학동네,2020-08-30,0,1,2020-09-04


In [28]:
data.loc[314]['tags']

"['학회', '스터디', 'IT', '취업']"

In [3]:
data.tag = data.tag.apply(ast.literal_eval)

In [4]:
#전체 태그 불러오기
alll = []
for _ in data.tag:
    alll+=_
alll = list(set(alll))
print(alll)

['공모전', '대외활동', '음악', '예술', '언어', '종교', '미술', '취업', '봉사', '운동', 'IT', '기타', '연합', '동아리', '사회', '학회', '경영', '친목', '과학', '스터디']


## Load User Table
=> 설문조사로 수집한 임의 유저 데이터

In [5]:
usertable = pd.read_csv('usertable.csv') 
usertable = usertable.iloc[:,1:]
usertable.head()

,userid,username,password,grade,college,sex
0,0,A,password,4학년,사범대학,M
1,1,B,password,2학년,경영대학,F
2,2,C,password,4학년,생명과학대학,M
3,3,D,password,1학년,국제학부,F
4,4,E,password,2학년,생명과학대학,M


In [6]:
i = usertable.loc[usertable.userid==1]
i.grade.iloc[0]

'2학년'

## Load View Table
=> 설문조사를 통해 특정 유저가 조회할만한 홍보물을 수집

In [7]:
viewtable = pd.read_csv('lookup.csv') 
viewtable = viewtable.iloc[:,1:]
viewtable.head()

,lookupid,userid,articleid,viewed,scrapped
0,0,0,487,2,1
1,1,0,665,2,1
2,2,0,97,2,1
3,3,0,590,2,1
4,4,0,810,2,1


## Preprocess Data - 1. 데이터 합병
- 유저별 조회수 합산  
- 홍보물별 조회수 합산

In [8]:
# 유저별 조회수 합산
userviewed = viewtable['viewed'].groupby(viewtable['userid']).sum()
userviewed

userid
0     12
1     30
2     26
3     33
4      8
5     21
6     11
7     69
8      6
9     17
10    33
11     2
12    12
13    18
14     4
15    58
16    12
17     2
18    50
19    52
20     4
21    12
22    39
23    69
24    75
25     9
26    18
27     4
28     2
29     6
30    21
Name: viewed, dtype: int64

In [9]:
usertable = pd.concat([usertable, userviewed],axis=1)
usertable.head()

,userid,username,password,grade,college,sex,viewed
0,0,A,password,4학년,사범대학,M,12.0
1,1,B,password,2학년,경영대학,F,30.0
2,2,C,password,4학년,생명과학대학,M,26.0
3,3,D,password,1학년,국제학부,F,33.0
4,4,E,password,2학년,생명과학대학,M,8.0


In [10]:
# 홍보물별 조회수 합산
artviewed = viewtable['viewed'].groupby(viewtable['articleid']).sum()
artviewed.head()

articleid
2     2
10    3
15    3
19    3
26    2
Name: viewed, dtype: int64

In [11]:
artviewed = artviewed.reset_index()
artviewed.head()

,articleid,viewed
0,2,2
1,10,3
2,15,3
3,19,3
4,26,2


In [12]:
data['viewed'] = data.merge(artviewed, how='left')['viewed'].fillna(0)
data.head()

,articleid,title,host,uploaddate,duedate,description,image,tag,uploaderid,offlineupload,viewed
0,1,한국야쿠르트 마케팅 공모전,한국야쿠르트,2020-09-04,2020-08-31,2020년 한국야쿠르트 마케팅 공모전 ■ 공모주제 · 언택트 시대에 걸맞은 차별...,https://s3.ap-northeast-2.amazonaws.com/media....,"[공모전, 경영]",3,1,0.0
1,2,행정안전부 제1회 도전.한국 대국민 아이디어 공모전,행정안전부,2020-09-04,2020-09-15,제1회 도전.한국 대국민 아이디어 공모전▷ 지원자격 : 관심 있는 국민‧기업‧단체...,https://s3.ap-northeast-2.amazonaws.com/media....,"[공모전, 봉사]",10,1,2.0
2,3,K-water 2020년 대국민 환경 빅데이터 공모전,k-water,2020-09-04,2020-08-31,2020년 K-water 대국민 환경 빅데이터 공모전▷ 담당자 전화번호 :042-6...,https://s3.ap-northeast-2.amazonaws.com/media....,"[공모전, IT, 과학]",2,1,0.0
3,4,기획재정부 국민참여예산 대학생 공모전,기획재정부,2020-09-04,2020-08-31,[기획재정부] 국민참여예산 대학생 공모전 (~8/31)■ 신청기간2020년 6월 2...,https://s3.ap-northeast-2.amazonaws.com/media....,"[공모전, 사회]",10,0,0.0
4,5,문학동네 2020 제26회 문학동네 소설상 공모,문학동네,2020-09-04,2020-08-30,2020 제26회 문학동네 소설상 공모▷ 응모마감 : 2020년 8월 30일 (마감...,NaN,[공모전],0,1,0.0


## Preprocess Data - 2. 홍보물 간 TFIDF matrix + Cosine Similarity  (For CBF)

In [52]:
#integrate title and description
data['txt'] = data['title']+data['description']
data['txt']

0      한국야쿠르트 마케팅 공모전2020년 한국야쿠르트 마케팅 공모전   ■ 공모주제 · ...
1      행정안전부 제1회 도전.한국 대국민 아이디어 공모전제1회 도전.한국 대국민 아이디어...
2      K-water 2020년 대국민 환경 빅데이터 공모전2020년 K-water 대국민...
3      기획재정부 국민참여예산 대학생 공모전[기획재정부] 국민참여예산 대학생 공모전 (~8...
4      문학동네 2020 제26회 문학동네 소설상 공모2020 제26회 문학동네 소설상 공...
                             ...                        
995    [기간연장]커리어코치 11기(모의면접관) 선발안내\n커리어코치 선발 지원 기간을 ~...
996    12/12(목) 모의PT면접(17:30-20:30)\n본 페이지는 12/12(목) ...
997    12/11(수) 모의토론면접(17:30-20:30)\n본 페이지는 12/11(수) ...
998    12/10(화) 모의PT면접(17:30-20:30)\n본 페이지는 12/10(화) ...
999    12/5(목) 모의PT면접(17:30-20:30)\n본 페이지는 12/5(목) PT...
Name: txt, Length: 1000, dtype: object

In [53]:
#특수문자 제거
def cleantxt(txt):
    if not pd.isnull(txt):
        txt = txt[:-35]
        txt = txt.replace('\n',' ')
        txt = txt.replace('\r','')
        txt = txt.replace('(',' ')
        txt = txt.replace(')',' ')
        p = re.compile(r'[A-Za-z가-힣?.!\'\ ]+')
        txt = ''.join(re.findall(p,txt))
        txt = re.sub(' +', ' ', txt).strip()
    return txt

data['txt'] = data['txt'].apply(cleantxt)
data['txt']

0      한국야쿠르트 마케팅 공모전년 한국야쿠르트 마케팅 공모전 공모주제 언택트 시대에 걸맞...
1      행정안전부 제회 도전.한국 대국민 아이디어 공모전제회 도전.한국 대국민 아이디어 공...
2      Kwater 년 대국민 환경 빅데이터 공모전년 Kwater 대국민 환경 빅데이터 공...
3      기획재정부 국민참여예산 대학생 공모전기획재정부 국민참여예산 대학생 공모전 신청기간년...
4      문학동네 제회 문학동네 소설상 공모 제회 문학동네 소설상 공모 응모마감 년 월 일 ...
                             ...                        
995    기간연장커리어코치 기 모의면접관 선발안내 커리어코치 선발 지원 기간을 금 오후 시까...
996    목 모의PT면접 본 페이지는 목 PT면접 신청 페이지입니다. 신청기간 각 일정일 전...
997    수 모의토론면접 본 페이지는 수 토론면접 신청 페이지입니다. 신청기간 각 일정일 전...
998    화 모의PT면접 본 페이지는 화 PT면접 신청 페이지입니다. 신청기간 각 일정일 전...
999    목 모의PT면접 본 페이지는 목 PT면접 신청 페이지입니다. 신청기간 각 일정일 전...
Name: txt, Length: 1000, dtype: object

In [7]:
#불용어 불러오기
stopwords = pd.read_pickle('stopwords.pkl')
stopwords = list(stopwords)

#tokenizing
from konlpy.tag import Komoran
komo = Komoran() #어미/어근의 분류가 잘 되어있다 판단.

class Tokenize:
    def __init__(self):
        pass
    def __call__(self, text):
        #stemming
        #어미 관련 모두 제외
        stemmed = [_ for _ in komo.pos(text) if _[1] not in ['EC', 'EF', 'EP', 'ETM', 'ETN']]
        
        #lemmatizing
        #VV=동사어근, VA=형용사어근
        lemma = [_[0]+'다' if (_[1]=='VA') or (_[1]=='VV') else _[0] for _ in stemmed]
        
        #remove stopwords
        tokens = [_ for _ in lemma if _ not in stopwords]
        
        return tokens
    

In [8]:
tokenize = Tokenize()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

#ngram, stopwords 기능 지원
#객체 생성
tfidf = TfidfVectorizer(tokenizer = tokenize,
                        analyzer = 'word', ngram_range=(1,2), 
                        min_df=0) #ngram called here

In [10]:
tfidf_matrix = tfidf.fit_transform(clubstxt)

In [11]:
tfidf_matrix.shape

(200, 24658)

In [12]:
from sklearn.metrics.pairwise import linear_kernel
#linear_kernel = dot product
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

sorted(cosine_sim[0],reverse=True)[1:10]

[0.9885866433178748,
 0.946963656606428,
 0.09408093366423495,
 0.08573957388650287,
 0.08434785808134902,
 0.08372156296017692,
 0.08368143958290032,
 0.08275697907426006,
 0.08253045494638345]

In [44]:
with open('cosine_sim.pkl','wb') as f:
    pickle.dump(cosine_sim, f)

↑새로운 데이터가 들어올 때 마다 cosine_similarity 업데이트 한다.  
tfidf & cosine similarity 계산하는 코드 따로 생성 필요

## Simple RS

In [13]:
def simpleRS(user_id):
    
    graded = {'1학년': ['동아리', '친목', '연합'],
              '2학년' : ['동아리','대외활동','봉사'],
              '3학년' : ['대외활동', '봉사'],
              '4학년' : ['학회', '스터디', '공모전', '취업']}


    sexd = {'F' : ['음악','예술', '언어'],
            'M' : ['운동', '음악', 'IT']}


    colleged = {'정보대학' : ['IT'],
                '정보보호학부' : ['IT'],
                '공과대학' : ['IT'],
                '이과대학' : ['IT'],
                '정경대학' :  ['경영', '대외활동'], 
                '경영대학' :  ['경영', '대외활동'], 
                '간호대학':['사회'], 
                '보건과학대학':['사회'],
                '국제학부' : ['언어'],
                '디자인조형학부' : ['예술'],
                '미디어학부' : ['경영', '대외활동', '예술', '음악'],
                '문과대학' : ['대외활동', '사회', '언어'] , 
                '사범대학' : ['대외활동', '사회', '언어'] ,
                '생명과학대학':['과학'], 
                '의과대학' : ['과학','봉사']}

    userinfo = usertable.loc[usertable.userid==user_id]
    
    selected = list(set(graded[userinfo.grade.iloc[0]]+sexd[userinfo.sex.iloc[0]]+colleged[userinfo.college.iloc[0]]))
    
    ## 사용자정보에 매핑하여 속하는 동아리 추출
    data_selected = data.loc[data.tag.apply(lambda y : any(x in y for x in selected))]
    
    ## 조회수별로 정렬
    selected_ordered = data_selected.sort_values(by='viewed', ascending=False)

    print('<학년:{}, 성별:{}, 단과대학:{}> 인 당신에게 어울리는 홍보물을 조회순으로 나열'.format(
        userinfo.grade.iloc[0],userinfo.sex.iloc[0],userinfo.college.iloc[0]))
    return selected_ordered.title.reset_index().title

In [30]:
simpleRS(1)

<학년:2학년, 성별:F, 단과대학:경영대학> 인 당신에게 어울리는 홍보물을 조회순으로 나열


0        [2020 YOUNG TIGERS 신입단원 2차 모집] 
1                      사단법인 청년과미래 정회원 모집
2             서대문장애인종합복지관 장애아동프로그램 봉사자모집
3                 월드투게더 온라인서포터즈 해피메신저 모집
4          ??고려대학교 밴드동아리 TRUSS 29기 모집 ? 
                     ...                
668             고려대학교 조정부 남자,여자 부원 모집 ? 
669    고려대학교 생명과학대학 축구동아리 ATGC 신입부원 모집~ 
670            늙은이를 환영하는 동아리가 있다?? 뿌슝빠슝 
671        고려대학교 중앙동아리 풍물패 고대농악대(KUNAD) 
672          ??체육교육과 FC드림 사진&영상 기자 모집?? 
Name: title, Length: 673, dtype: object

## CBF
조회했던 홍보물을 모두 모아 각 홍보물-나머지 홍보물들에 대한 cosine similarity를 구한다. 그 중 가장 cosine similarity가 높은것부터 나열

In [8]:
import pickle
import pandas as pd
with open('cosine_sim.pkl','rb') as f:
        cosine_sim = pickle.load(f)

In [13]:
pd.Series(cosine_sim[20]).sort_values(ascending=False)[1:]

121    0.180002
174    0.147396
33     0.146640
110    0.139188
114    0.132134
         ...   
895    0.000000
685    0.000000
294    0.000000
896    0.000000
894    0.000000
Length: 999, dtype: float64

In [129]:
#viewtable
def cbf(userId,no_recom=10):
    #index에대한 제목
    names = data.title
    
    #조회한 동아리들에대한 cosine_sim 모두 더하기(조회수 2회이상이면 그만큼 곱하기)
    sim_scores=np.zeros(len(cosine_sim[0]))
    for n,i in enumerate(viewtable[viewtable['userid']==userId]['articleid']):
        sim_scores += (np.array(cosine_sim[i])*viewtable[viewtable['userid']==userId].reset_index()['viewed'][n])
    
    sim_scores = list(enumerate(sim_scores))
    #sim 높은순부터 정렬
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
    res = names[[s[0] for s in sim_scores]]
    tmp = []
    i=0
    while(len(tmp)<no_recom):
        if res.iloc[i] not in tmp:
            tmp.append(res.loc[i])
        i+=1S
    print('당신이 조회했던 홍보물과 유사한 홍보물을 추천')
    return pd.Series(tmp)

In [93]:
cbf(2)

당신이 조회했던 홍보물과 유사한 홍보물을 추천


0                          한국야쿠르트 마케팅 공모전
1            행정안전부 제1회 도전.한국 대국민 아이디어 공모전
2           K-water 2020년 대국민 환경 빅데이터 공모전
3                    기획재정부 국민참여예산 대학생 공모전
4              문학동네 2020 제26회 문학동네 소설상 공모
5    Korea Mobile Fest 슬기로운 방구석 생활 영상 공모전
6                        대한두통학회 두통 이야기 공모
7           식품의약품안전처 2020 나트륨·당류줄이기UCC공모전
8      브이로그(V-log) in 포천(Pocheon) 마케팅 공모전
9                        금융감독원 제15회 금융공모전
dtype: object

# CF(Matrix Factorization)

In [98]:
user_item = viewtable.pivot_table(values = 'viewed', index = 'userid', columns = 'articleid',dropna=False)
user_item=user_item.fillna(0) #nan값 대신 0 으로 채우기
user_item.head()

articleid,2,10,15,19,26,28,31,46,56,57,...,853,854,860,862,864,867,869,870,872,873
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
from numpy.linalg import svd
from scipy.sparse.linalg import svds #does truncated svd

U, sigma, Vt = svd(user_item)
#find cumsum
cumratio = list(map(lambda x : round(x,3), np.cumsum(sigma)/np.sum(sigma)))

#80%의 설명력을 가지고 있는 요소들만 pick
toleaveout = len(list(filter(lambda x : x<0.8, cumratio)))
toleaveout

17

In [100]:
U, sigma, Vt = svds(user_item, k = toleaveout) #k=남길 특이값 개수

sigma = np.diag(sigma)

#convert back to original matrix by matrix multiplaction
svd_pred = np.dot(np.dot(U,sigma),Vt)
svd_pred = pd.DataFrame(svd_pred)
svd_pred #값이 달라짐 - truncated를 썼기 때문

,0,1,2,3,4,5,6,7,8,9,...,274,275,276,277,278,279,280,281,282,283
0,-6.956458e-03,2.145354e-03,2.145354e-03,-8.257389e-03,-6.956458e-03,-2.149288e-02,-2.149288e-02,-1.962897e-02,1.080241e-02,2.145354e-03,...,2.145354e-03,-6.087475e-03,-4.493948e-17,-1.058500e-03,-1.058500e-03,2.145354e-03,-1.670023e-03,1.636120e-03,2.145354e-03,-3.488730e-02
1,-5.027598e-03,1.091612e-03,1.091612e-03,-4.714519e-03,-5.027598e-03,-1.401978e-02,-1.401978e-02,-8.572646e-03,-3.143967e-03,1.091612e-03,...,1.091612e-03,-5.262711e-03,-1.114212e-16,8.071791e-04,8.071791e-04,1.091612e-03,2.351382e-04,2.681974e-03,1.091612e-03,3.781438e-02
2,-1.754851e-03,5.290506e-04,5.290506e-04,-6.142286e-03,-1.754851e-03,1.985429e+00,1.985429e+00,1.843759e-03,4.584811e-02,5.290506e-04,...,5.290506e-04,-4.124780e-03,-1.437454e-17,2.462464e-03,2.462464e-03,5.290506e-04,8.317934e-04,3.254303e-03,5.290506e-04,2.031710e+00
3,-1.317898e-03,2.915258e-04,2.915258e-04,-1.098941e-03,-1.317898e-03,1.641643e-03,1.641643e-03,-4.067049e-04,-1.891054e-03,2.915258e-04,...,2.915258e-04,1.013415e-03,1.548447e-16,2.993215e+00,2.993215e+00,2.915258e-04,4.047716e-05,-7.808380e-05,2.915258e-04,-4.452102e-03
4,5.718674e-16,2.268001e-16,2.268001e-16,-1.415645e-16,5.718674e-16,1.484224e-16,1.484224e-16,3.255012e-16,5.336827e-18,2.268001e-16,...,2.268001e-16,1.261002e-16,-1.530184e-16,3.851788e-17,3.851788e-17,2.268001e-16,-2.607537e-19,8.776783e-16,2.268001e-16,1.732853e-16
5,5.879243e-04,-2.918284e-05,-2.918284e-05,1.066105e-04,5.879243e-04,-5.034073e-05,-5.034073e-05,-2.144804e-05,-1.222176e-04,-2.918284e-05,...,-2.918284e-05,4.594066e-05,-5.615811e-18,-1.664665e-06,-1.664665e-06,-2.918284e-05,6.389628e-05,2.999772e+00,-2.918284e-05,-7.433347e-03
6,3.153892e-02,-6.874568e-03,-6.874568e-03,6.989406e-02,3.153892e-02,9.169622e-02,9.169622e-02,-7.495975e-03,5.002823e-03,-6.874568e-03,...,-6.874568e-03,-5.413782e-03,-3.114908e-16,-5.673162e-03,-5.673162e-03,-6.874568e-03,3.761870e-04,-1.807240e-02,-6.874568e-03,9.423761e-02
7,1.800065e-05,3.075834e-05,3.075834e-05,-6.436462e-04,1.800065e-05,-2.749853e-03,-2.749853e-03,5.380757e-04,-1.804594e-03,3.075834e-05,...,3.075834e-05,2.998699e+00,-1.706011e-16,1.013415e-03,1.013415e-03,3.075834e-05,-4.754746e-04,6.827040e-04,3.075834e-05,1.202706e-02
8,-2.655216e-17,-6.351767e-16,-6.351767e-16,-2.797442e-16,-2.655216e-17,-2.368786e-17,-2.368786e-17,1.102960e-16,-3.592608e-18,-6.351767e-16,...,-6.351767e-16,-7.248088e-17,3.352732e-16,-3.984397e-16,-3.984397e-16,-6.351767e-16,9.110332e-19,-4.776459e-16,-6.351767e-16,-1.610102e-17
9,-1.725688e-02,-2.683084e-03,-2.683084e-03,3.384314e-02,-1.725688e-02,1.028353e-01,1.028353e-01,-1.496059e-02,6.760155e-03,-2.683084e-03,...,-2.683084e-03,3.879600e-02,-3.644470e-16,-6.458686e-03,-6.458686e-03,-2.683084e-03,4.806360e-04,-1.286483e-02,-2.683084e-03,1.079444e-01


In [128]:
def cf(user_id, num_recom=10):
    names = data.title
    
    #sort expected ratings
    sorted_index = svd_pred.loc[user_id].sort_values(ascending=False).index
    
    #get name list of clubs seen by user
    seen_clubs = svd_pred.columns[user_item.loc[user_id]!=0]
    
    #get index list of clubs seen by user
    seen_names = user_item.loc[user_id][user_item.loc[user_id]!=0].index
    seen_names = list(seen_names.unique())

    #get top n from movies user has not seen
    sorted_notseen = [x for x in sorted_index if x not in seen_clubs]
    recom_title = list(user_item.columns[sorted_notseen][:num_recom])

    print('what you have seen : \n{}'.format(names[seen_names]))
    print()
    print('Recommended for you : \n')
    return names[recom_title]

In [125]:
cf(20)

what you have seen : 
219                         청음복지관 2020년 청음 멘토링 열~정 멘토 모집
638                                 ♬고려대학교 합창단♬ 신입단원 모집 
691    고려대학교 농구동아리 '서우회' 에서 2020년도 신입부원을 모집합니다 !! (남자...
869    [오늘 8시 마감]??고려대학교 유일의 브랜드 컨설팅 학회 CREAKET에서 14기...
Name: title, dtype: object

Recommended for you : 



211                                  2020 스파오 서포터즈 2기 모집
395                             [KUSPA] ?그땐 KU랬지?? 요모조모 
642                                 고려대학교 조정부 신입부원 모집 ? 
697    ??D-day?? 고려대학교 소속 유일한 법학회!! 고려대학교 사이버법정책학회(CL...
797    [설명회 자료 추가] 고려대학교 소속 유일한 법학회!! 고려대학교 사이버법정책학회(...
606                                코로나 연구 참여자 모집 [UAEM] 
743          고려대 최초?최다수상 토론/스피치 동아리 ?코기토? 회원모집 (온라인 면접) 
274                                 하남시 긴급재난 서포터즈 봉사단 모집
787           [마감 D-5] 학내 유일 중앙토론동아리 고란도란에서 15기를 모집합니다! 
824                          고려대학교 영화제작동아리 돌빛 2020 리쿠르팅 
Name: title, dtype: object

# 종합

#### USER PHASE
phase1) 초기상태(조회 10 미만)  
- simple recommender*1  
  
phase2) 조회 10 이상  
- simple*0.1 + content based*0.6 + collaborative*0.3  
  
phase3) 조회 20 이상  
- simple*0.1 + content based*0.4 + collaborative*0.5  


In [115]:
simpleRS(3)

<학년:1학년, 성별:F, 단과대학:국제학부> 인 당신에게 어울리는 동아리의 홍보물을 조회순으로 나열


0                        [2020 YOUNG TIGERS 신입단원 2차 모집] 
1                          ??고려대학교 밴드동아리 TRUSS 29기 모집 ? 
2            [Piano] ?? 고려대학교 중앙 피아노 동아리 TTP 신입부원 모집 ?? 
3      [D-2]??[고집] 고려대학교 해비타트 건축봉사 CCYP 동아리 고집에서 10기 ...
4                   [스쿼시] 고려대학교 중앙 스쿼시동아리 ?엔더스? 신입부원 모집 
                             ...                        
467                    ○ 중앙동아리 예술비평연구회에서 신입회원을 모집합니다! ○ 
468                                           “편안해지다” ? 
469                 [스쿼시] 고려대학교 중앙 스쿼시동아리 ?엔더스? 신입부원 모집 
470               ?[기독교/중앙동아리] 예수전도단 예배영상 / 여러분을 초대합니다? 
471    (D-1) [KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을...
Name: title, Length: 472, dtype: object

In [116]:
cbf(3)

당신이 조회했던 홍보물과 유사한 홍보물을 추천


0                          한국야쿠르트 마케팅 공모전
1            행정안전부 제1회 도전.한국 대국민 아이디어 공모전
2           K-water 2020년 대국민 환경 빅데이터 공모전
3                    기획재정부 국민참여예산 대학생 공모전
4              문학동네 2020 제26회 문학동네 소설상 공모
5    Korea Mobile Fest 슬기로운 방구석 생활 영상 공모전
6                        대한두통학회 두통 이야기 공모
7           식품의약품안전처 2020 나트륨·당류줄이기UCC공모전
8      브이로그(V-log) in 포천(Pocheon) 마케팅 공모전
9                        금융감독원 제15회 금융공모전
dtype: object

In [117]:
cf(3)

what you have seen : 
361                    고려대학교 생명과학대학 축구동아리 ATGC 신입부원 모집~ 
365            대학연합독서동아리?? 빛나는 책갈피??의 11기 신입회원을 모집합니다!! 
425                                 [고려대학교 역도부] 신입부원 모집 
461                      [마음공부] 이제는 밝은 세상! 마음을 밝히는 방법 ? 
495                                           “편안해지다” ? 
497                    ○ 중앙동아리 예술비평연구회에서 신입회원을 모집합니다! ○ 
503    [금주토요일마감]고려대학교 정치외교학회 POLITIKA 2020 1학기 리쿠르팅!!...
544                    고려대학교 생명과학대학 축구동아리 ATGC 신입부원 모집~ 
571                      진보정치시사연합동아리 <시사톡> 동아리원을 모집합니다! 
862                ??집수리봉사동아리 KU-HOPE 2020년 1학기 리크루팅!?? 
864                           [KULSOM] 대륙의 거대기업 파헤치기!?? 
Name: title, dtype: object

Recommended for you : 
626                           신종코로나 바이러스 사태로 인한 서화회 FAQ 
649    [일정 변경]??고려대학교 유일의 유튜브 미디어 동아리 “쿠리에이터”에서 4기 신입...
682                   포근하고 편안한 원불교학생회로 오세요! (누구나 환영!) ? 
434                             ??극예술연구회 신입생 추가 모집 안내?? 
272                            2020년 환자안전 온라인 서포터즈 모집 안내
402            

[626, 649, 682, 434, 272, 402, 222, 543, 522, 636]

In [133]:
def recommendation_system(user_id):
    userinfo = usertable.loc[usertable.userid==user_id]
    notfirsttime=True
    
    if userinfo.viewed.iloc[0]<10:
        notfirsttime=False
        ratio = [1,0,0]
        
    elif userinfo.viewed.iloc[0]<20: ratio=[0.1,0.6,0.3]
    else: ratio=[0.1,0.3,0.6]
    
    
    result = dict()

    #simpleRS
    SRSres = simpleRS(user_id)
    for n,_ in enumerate(SRSres):
        if _ not in result.keys():
            result[_]=ratio[0]*(abs(n-len(SRSres))/len(SRSres)) #minmaxscalar inverted (ranking에 점수주기)
            
    #when 조회 less than 10, do not apply cbf and cf
    if notfirsttime:
        #cbf
        for _ in cbf(user_id):
            if _ in result.keys():
                result[_]+=ratio[1]
            else:
                result[_]=ratio[1]

        #cf
        for _ in cf(user_id):
            if _ in result.keys():
                result[_]+=ratio[2]
            else:
                result[_]=ratio[2]

    #sort by weight
    result = sorted(result.items(), key=lambda x: x[1],reverse=True)
    recommend = list([_[0] for _ in result])
    return recommend[1:20]

In [134]:
recommendation_system(3)

['고려대학교 유일의 중앙 통기타 동아리 그루터기 신입 부원 모집 ',
 '신종코로나 바이러스 사태로 인한 서화회 FAQ ',
 '[일정 변경]??고려대학교 유일의 유튜브 미디어 동아리 “쿠리에이터”에서 4기 신입 부원을 모집합니다! ',
 '포근하고 편안한 원불교학생회로 오세요! (누구나 환영!) ? ',
 '[마음공부] 이제는 밝은 세상! 마음을 밝히는 방법 ? ',
 '??극예술연구회 신입생 추가 모집 안내?? ',
 '[어갑배] 고려대학교 배드민턴 소모임 어갑배에서 신입회원을 모집합니다! ',
 '2020년 환자안전 온라인 서포터즈 모집 안내',
 '한국심장재단 대학생기자단 4기 모잡',
 '한국야쿠르트 마케팅 공모전',
 '행정안전부 제1회 도전.한국 대국민 아이디어 공모전',
 'K-water 2020년 대국민 환경 빅데이터 공모전',
 '기획재정부 국민참여예산 대학생 공모전',
 '문학동네 2020 제26회 문학동네 소설상 공모',
 'Korea Mobile Fest 슬기로운 방구석 생활 영상 공모전',
 '대한두통학회 두통 이야기 공모',
 '식품의약품안전처 2020 나트륨·당류줄이기UCC공모전',
 '브이로그(V-log) in 포천(Pocheon) 마케팅 공모전',
 '금융감독원 제15회 금융공모전']